In [28]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option("display.max_columns", None)

In [30]:

# lee mobiliarios
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")

# Lee equipos medicos y los ordena
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")
df_equipos_medicos = df_equipos_medicos.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")
df_industriales = df_industriales.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# Lee Informaticos
df_informaticos = pd.read_csv("../data/processed/df_procesada_informaticos.csv")

In [31]:
union_bienes_con_informatico = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)

# Deja solamente las columnas utiles
columnas_utiles = [
    "correlativo_antiguo",
    "n_inventario_2025",
    "bien",
    "marca",
    "modelo",
    "serie",
    "tipo_bien",
    "piso",
    "unidadservicio_clinico",
    "ubicacion_unidad",
    "propiedad",
    "observaciones",
]

# Agrega el conteo de bienes
union_bienes_con_informatico["conteo"] = 1

conteo_equipos = pd.pivot_table(
    union_bienes_con_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

In [32]:
unidades_a_preguntar = [
    "4 NORTE",
    "CAMAS MEDIAS RESPIRATORIO",
    "ENFERMERA COORDINADORA",
    "FARMACIAS",
    "JEFE MEDICO CARDIOCIRUGIA",
    "MEDICINA RESPIRATORIA",
    "MQ",
    "RECEPCION DE MUESTRA",
    "SALA CLINICA 3ERO SUR",
    "SALA DE MEDICOS CARDIOCIRUGIA",
    "SALA MEDICOS",
    "SALA RESIDENTE 3RO NORTE",
    "SECRETARIA CARDIOCIRUGIA",
    "SECRETARIA ENFERMERIA UTI 2",
    "UCHILE",
    "UCI 5",
    "UNIDAD PACIENTE CRITICO 3",
    "UPC",
    "UCI-UPC",
    "UCI-UTI",
    "UPC 3",
    "UPC 4",
    "UPC 5 PISO",
    "UPC-UCI",
    "UTI 4",
    "UCI 5 NORTE",
]

## Asignacion de Correlativos

Se hara de forma separada

In [33]:
GLOSAS_CON_CORRELATIVOS = [
    "INT",
    "INT (SSMO)",
    "INT (MEL)",
    "INT (MINSAL)",
    "INT (DONACION)",
    "COMODATO",
]

In [34]:
# Asigna correlativos a equipos medicos
df_equipos_medicos, ultimo_correlativo_eq_med = fa.asignar_correlativos(
    df_equipos_medicos, 4016, GLOSAS_CON_CORRELATIVOS, "n_inventario_2025"
)

# Asigna correlativos a industriales
df_industriales, ultimo_correlativo_ind = fa.asignar_correlativos(
    df_industriales, ultimo_correlativo_eq_med + 1, GLOSAS_CON_CORRELATIVOS, "n_inventario_2025"
)

# Asigna correlativos a informaticos
df_informaticos, ultimo_correlativo_inf = fa.asignar_correlativos(
    df_informaticos, ultimo_correlativo_ind + 1, GLOSAS_CON_CORRELATIVOS, "n_inventario_2025"
)

In [ ]:
# Genera la matriz total de bienes
matriz_total_correlativos = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)

orden_columnas = [
    "correlativo_antiguo",
    "n_inventario_2025",
    "bien",
    "marca",
    "modelo",
    "serie",
    "tipo_bien",
    "piso",
    "unidadservicio_clinico",
    "ubicacion_unidad",
    "propiedad",
    "observaciones",
]

matriz_total_correlativos = matriz_total_correlativos[orden_columnas]

In [36]:
# Exporta asignando correlativos
df_mobiliario.to_excel("../data/interim/1_matriz_mobiliarios_2025.xlsx", index=False)
df_equipos_medicos.to_excel("../data/interim/2_matriz_equipos_medicos_2025.xlsx", index=False)
df_industriales.to_excel("../data/interim/3_matriz_industriales_2025.xlsx", index=False)
df_informaticos.to_excel("../data/interim/4_matriz_informaticos_2025.xlsx", index=False)

In [37]:
fa.guardar_dataframe_como_tabla_excel(
    matriz_total_correlativos,
    "../data/interim/matriz_bienes_INT_2025.xlsx",
    nombre_tabla="MatrizBienes",
    estilo_tabla="TableStyleMedium1"
)

✅ Archivo guardado exitosamente en: ../data/interim/matriz_bienes_INT_2025.xlsx


## Exportar bases para etiquetas

In [38]:
# Aisla la matriz de equipos medicos
for unidad in matriz_total_correlativos.query("tipo_bien == 'EQUIPO MEDICO'")[
    "unidadservicio_clinico"
].unique():
    # Aisla la unidad
    df_equipos_medicos_unidad = df_equipos_medicos.query("unidadservicio_clinico == @unidad")

    fa.guardar_dataframe_como_tabla_excel(
        df_equipos_medicos_unidad,
        f"../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS {unidad}.xlsx",
        nombre_tabla="MatrizBienes",
        estilo_tabla="TableStyleMedium1",
    )

✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MEDICINA FISICA Y REHABILITACION.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS CONSULTORIO EXTERNO.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS LABORATORIO.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS UNIDAD DE APOYO FARMACIA.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS FUNCION PULMONAR.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS UMT.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MQ 3 NORTE.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MQ 3 SUR.xl